# Déploiement d'une application web sur SSP Cloud

Dans ce tutoriel, on explore le processus de déploiement d'une application web sur le SSP Cloud. Pour ce faire, on va suivre les étapes suivantes:
- Conteneurisation de l'application
- Déploiement sur SSP Cloud en utilisant un Helm Chart

On s'intéresse particulièrement aux applications Python et R.
Ce 

## 1- Conteneurisation

Pour pouvoir être déployée sur un cluster *Kubernetes*, une application doit nécessairement être mise à disposition sous la forme d'une image *Docker*. Concrètement, cette étape permet de rendre l'application portable : une fois que l'image Docker est construite et fonctionne correctement, elle peut être déployée sur n'importe quel environnement d'éxécution avec la certitude qu'elle fonctionnera de manière attendue, peu importe l'environnement qui a servi à la développer.

Le fichier Dockerfile situé à la racine du projet contient une suite d'instructions qui permettent de conteneuriser l'application, sous la forme d'une image Docker.

Dans cet premier exemple, on crée un Dockerfile pour une application web développée en Python avec Flask (le code source est disponible [**ici**](https://github.com/amine-bs/flask-tutorial)).

*<span style='color:RED'> FROM  </span> inseefrlab/onyxia-python-minimal:latest 
<br>
<span style='color:RED'> COPY  </span> app ./
<br>
<span style='color:RED'> RUN  </span> pip install -r requirements.txt
<br>
<span style='color:RED'> EXPOSE  </span> 5000
<br>
<span style='color:RED'> ENTRYPOINT  </span> ["flask", "--app", "main", "run", "--host", "0.0.0.0", "-p", "5000"]*

Le fichier contient 5 parties:
- appel de l'image Docker de base: ``inseefrlab/onyxia-python-minimal``: Il s'agit d'une image de base contenant Python, les librairies systèmes, et les packages nécessaires au fonctionnement des applications Python. Elle est maintenue dans le cadre du projet Onyxia.
- Copier le code de l'application dans l'image: On copie le code de l'application qui est dans le dossier ``app`` dans le directoire du travail de l'image.
- Installation de dépendances de l'application dans l'image
- Exposer le port utilisé par l'application: les applications Flask utilisent par défault le port 5000.
- Entrypoint: c'est la commande de lancement du conteneur. Ici, ``main`` fait référence au fichier ``main.py``

Dans cet deuxième exemple, on suit les mêmes étapes pour créer un Dockerfile pour une application développée en R avec Shiny (le code source est disponible [**ici**](https://github.com/InseeFrLab/template-shiny-app)).

une fois l'image créée, nous devons la pousser vers un référentiel. Docker hub offre des dépôts publics qu'on peut utiliser gratuitement. Le push peut se faire avec la ligne de commande de Docker : docker push image. Mais c'est recommandé de mettre en place une intégration continue pour mettre à jour l'image automatiquement.*<span style='color:RED'> FROM  </span> inseefrlab/onyxia-r-minimal
<br>
#Install required linux librairies
<br>
<span style='color:RED'> ROOT  </span> root
<br>
<span style='color:RED'> RUN  </span> apt-get update -y && \
    apt-get install -y --no-install-recommends libpq-dev \ 
                                               libssl-dev \
                                               libxml2-dev \
                                               gdal-bin \
                                               libgdal-dev
<br>
#Copy code
<br>
<span style='color:RED'> COPY  </span> myshinyapp/ ./myshinyapp
<br>
#Install dependencies
<br>
<span style='color:RED'> RUN  </span> Rscript -e 'remotes::install_deps("./myshinyapp")'
<br>
<span style='color:RED'> RUN  </span> Rscript -e 'install.packages("./myshinyapp", repos = NULL, type="source")'
<br>
#Expose port where shiny app will broadcast
<br>
<span style='color:RED'> EXPOSE  </span> 3838
<br>
<span style='color:RED'> RUN  </span> echo "local({options(shiny.port = 3838, shiny.host = '0.0.0.0')})" >> /usr/local/lib/R/etc/Rprofile.site
<br>
<span style='color:RED'> CMD  </span> ["Rscript", "-e", "myshinyapp::runApp()"]*

Une fois l'image créée, nous devons la pousser vers un dépôt. Docker hub offre des dépôts publics qu'on peut utiliser gratuitement. Le publication de l'image peut se faire avec la ligne de commande de Docker : ``docker push image``. Mais c'est recommandé de mettre en place une configuration d'intégration continue pour mettre à jour l'image automatiquement. Le tutoriel Shiny explique en détail l'intégration continue.

## 2- Déploiement sur SSP CLoud

### Création d'un Chart Helm

Le déploiement de l'application nécessite la création d'un chart Helm. Concrètement, un chart Helm peut être vu comme un package Kubernetes, contenant les ressources nécessaires au déploiement d'une application.

Le dépôt de [déploiement] contient un chart Helm permettant le déploiement de l'application template. Ce repository va servir de base pour le chart Helm de votre application.

In [5]:
!git clone https://github.com/amine-bs/helm-chart-web-deployment

Cloning into 'helm-chart-web-deployment'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 16 (delta 0), reused 13 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), 5.62 KiB | 958.00 KiB/s, done.


Le fichier ``values.yaml`` contient précisément les valeurs que l'on modifie. Les modifications à apporter dépendent naturellement de ce que réalise en pratique l'application, car cela conditionne les ressources dont elle a besoin. Dans un premier temps, il nous faut modifier :
- le chemin et nom de l'image
- le tag de l'image. Il s'agit du tag de l'image sur le DockerHub. Par défault, et pendant la phase de développement, on peut indiquer le tag latest pour signifier "la dernière version de l'image qui a été produite".
- l'hostname de l'Ingress, i.e. l'URL à laquelle l'application sera accessible une fois déployée. Cette URL doit être de la forme *.lab.sspcloud.fr ; par exemple dans notre cas : flask-tuto.lab.sspcloud.fr.

### Utilisation du stockage de données S3 avec MinIO
Si votre application n'utilise pas de données externes, ou contient ses propres données dans l'image Docker, vous pouvez donner la valeur false au paramètre ``s3.enabled`` et passer cette section. A l'inverse, si l'application utilise des données en entrée stockées sur ``MinIO``, il faut donner la valeur true au paramètre et fournir à l'application les informations d'authentification au service de stockage. Ces informations sont sensibles, et ne doivent donc jamais figurer en clair dans le code source de l'application ou sur un dépôt GitHub. Pour éviter ce risque, on va inscrire ces informations dans un objet Kubernetes appelé ``Secret``, qui va nous permettre de les passer à l'application sous la forme de variables d'environnement.

La première étape est de créer un compte de service sur la console MinIO. Pour ce faire :

- menu "Identity" -> "Service Accounts" -> "Create Service Account" -> "Create"
- comme précédemment, conserver à l'écran les informations de connexion.

La seconde étape est de créer un Secret Kubernetes contenant ces informations. Pour être accessible dans l'application, ce secret doit être appliqué comme une ressource dans le namespace Kubernetes dans lequel sera déployé l'application. 

Pour cela :
- Écrire le template suivant dans un fichier quelconque.yaml :

```yaml
apiVersion: v1
kind: Secret
metadata:
  name: myshinyapp-s3
type: Opaque
stringData:
  AWS_ACCESS_KEY_ID: changeme
  AWS_SECRET_ACCESS_KEY: changeme
  AWS_S3_ENDPOINT: minio.lab.sspcloud.fr
  AWS_DEFAULT_REGION: us-east-1
```

- Les valeurs de ``AWS_ACCESS_KEY_ID`` et ``AWS_SECRET_ACCESS_KEY`` sont à remplacer par les valeurs obtenues à l'étape précédente sur la console MinIO. Les valeurs de ``AWS_S3_ENDPOINT`` et ``AWS_DEFAULT_REGION`` n'ont pas besoin d'être modifiées pour une utilisation sur le cluster. Enfin, le nom du Secret (variable metadata.name) doit porter la même valeur que la variable ``s3.existingSecret``

Pour être accessible dans l'application, ce secret doit être appliqué comme une ressource dans le namespace Kubernetes dans lequel sera déployé l'application. Pour cela :

- mettre le template de secret dans un fichier quelconque.yaml et remplacer les valeurs comme indiqué ci-dessus
- dans un terminal, exécuter ``kubectl apply -f quelconque.yaml``
- si tout a bien fonctionné, un message devrait confirmer la création du secret. Du style secret/nom_de_secret created où nom_de_secret est ce que vous avez renseigné dans metadata.name du fichier quelconque.yaml.

Une fois le secret appliqué, les quatre variables d'environnement définies dans le secret sont accessibles dans l'application. Vu que ces variables sont standards, il est alors possible de se connecter au stockage MinIO via le package R ``aws.s3`` ou le package Python ``boto3`` sans même avoir besoin de les préciser

### Utilisation d'une base de données PostgreSQL
Si votre application n'utilise pas de base PostgreSQL, vous pouvez donner la valeur false au paramètre ``postgresql.enabled`` et passer cette section. Sinon, il faut donner la valeur true au paramètre ``postgresql.enabled``. Il est par ailleurs possible de changer les paramètres ``postgresql.username`` (nom d'utilisateur), ``postgresql.database`` (nom de la base de données) et ``postgresql.fullnameOverride`` (nom de domaine du service PostgreSQL) à sa guise, sachant que ces paramètres seront de toute manière passés automatiquement à l'application sous forme de variables d'environnement.

Les mots de passe de connexion, données sensibles, doivent quant à eux être passés à l'application via un Secret Kubernetes. La procédure est la même que précédemment, et le template de Secret à utiliser est :

```yaml
apiVersion: v1
kind: Secret
metadata:
  name: myshinyapp-postgresql
type: Opaque
stringData:
  password: changeme
  postgres-password: changeme
  replication-password: changeme
```

Trois passwords sont nécessaires, mais seul le champ ``stringData.password`` (password utilisateur) sera utilisé en pratique dans l'application. Il est donc possible de fixer le même password pour les trois champs de ``stringData`` sans trop de risque. Là encore, toutes ces informations (valeurs du chart et secrets) seront passées à l'application sous la forme de variables d'environnement

### Déploiement du Chart Helm
Finalement, pour déployer l'application sur le cluster à partir du terminal d'un service VSCode :

- cloner le dépôt contenant le chart de votre application (pas le template)
- importer les dépendances avec la commande ``helm dependency update chemin_du_depot_chart``
- installer le chart Helm : ``helm install chemin_du_depot_chart --generate-name``
Si tout a fonctionné, un message devrait confirmer l'instanciation du chart. On peut vérifier que ce dernier a bien été déployé avec la commande ``helm ls``, qui liste l'ensemble des instances de Chart helm déployées sur le cluster.